# <center>Alzheimer's Disease Prediction</center>

Alzheimer's disease is a progressive neurologic disorder that causes the brain to shrink (atrophy) and brain cells to die. Alzheimer's disease is the most common cause of dementia â€” a continuous decline in thinking, behavioral and social skills that affects a person's ability to function independently.







In this notebook, We are going to predict which class the person belongs to from the below 4 classes. We are going to use MRI scanned images as input for our CNN and predict the output.

The data consists of MRI images. The data has four classes of images both in training as well as a testing set:

1. Mild Demented

2. Moderate Demented

3. Non Demented

4. Very Mild Demented

# Importing Libraries

In [1]:
pip install -q -U keras-tuner

Note: you may need to restart the kernel to use updated packages.


In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Image Argumentation

In [3]:
train_datagen = ImageDataGenerator(
                    rescale = 1/255,
                    shear_range = 0.2,
                    zoom_range = 0.2,
                    horizontal_flip = True,
                    vertical_flip = False)
training_set = train_datagen.flow_from_directory('./Alzheimer_s Dataset/train',
                                                 target_size = (64, 64),
                                                 class_mode = 'categorical')

Found 5121 images belonging to 4 classes.


In [4]:
test_datagen = ImageDataGenerator(rescale = 1/255)
test_set = test_datagen.flow_from_directory('./Alzheimer_s Dataset/test',
                                            target_size = (64, 64),
                                            class_mode = 'categorical')

Found 1279 images belonging to 4 classes.


In [5]:
training_set.class_indices

{'MildDemented': 0,
 'ModerateDemented': 1,
 'NonDemented': 2,
 'VeryMildDemented': 3}

# Building the CNN

In [6]:
import keras

In [13]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(64,64,1)
    ),
    keras.layers.MaxPooling2D(pool_size=(2,2)),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.MaxPooling2D(pool_size=(2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(4, activation='softmax')
  ])
  
  model.compile(optimizer=tf.keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [8]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

C:\Users\sanja\AppData\Local\Temp/ipykernel_18140/556418634.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import RandomSearch


In [10]:
import keras_tuner as kt

In [14]:
tuner = kt.Hyperband(build_model,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='Alzheimers Disease')

In [18]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [24]:
tuner.search(training_set, epochs=50, validation_data = training_set, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
32                |?                 |conv_1_filter
5                 |?                 |conv_1_kernel
32                |?                 |conv_2_filter
3                 |?                 |conv_2_kernel
96                |?                 |dense_1_units
0.01              |?                 |learning_rate
2                 |?                 |tuner/epochs
0                 |?                 |tuner/initial_epoch
2                 |?                 |tuner/bracket
0                 |?                 |tuner/round

Epoch 1/2


UnimplementedError: Graph execution error:

Detected at node 'sequential/conv2d/Relu' defined at (most recent call last):
    File "C:\Users\sanja\anaconda3\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\sanja\anaconda3\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\sanja\anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "C:\Users\sanja\anaconda3\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "C:\Users\sanja\anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 677, in start
      self.io_loop.start()
    File "C:\Users\sanja\anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "C:\Users\sanja\anaconda3\lib\asyncio\base_events.py", line 596, in run_forever
      self._run_once()
    

In [ ]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

## Importing the Libraries

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense

## Initialising the CNN

In [6]:
cnn = Sequential()

## Step 1 - Convolution

In [7]:
cnn.add(Convolution2D(filters=32, kernel_size=3, activation='relu', input_shape=(64,64,1)))

## Step 2 - Pooling

In [8]:
cnn.add(MaxPooling2D(pool_size=(2,2)))

## Adding a second convolutional layer

In [9]:
cnn.add(Convolution2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(MaxPooling2D(pool_size=(2,2)))

## Step 3 - Flattening

In [10]:
cnn.add(Flatten())

## Step 4 - Adding Hidden Layers

In [12]:
from tensorflow.keras.layers import Dropout

In [13]:
## Hidden Layer-1
cnn.add(Dense(units=256, activation='relu'))
## Adding a Dropout Layer
cnn.add(Dropout(0.3))
## Hidden Layer-2
cnn.add(Dense(units=128, activation='relu'))

## Step 5 - Output Layer

In [14]:
cnn.add(Dense(units=4, activation='softmax'))

# Training the CNN

## Compiling the CNN

In [15]:
cnn.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

### Early Stopping

In [16]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0.0001,
    patience=20,
    verbose=1,
    mode="auto",
    baseline=None,
    restore_best_weights=False,
)

## Training the CNN on the Training set and evaluating it on the Test set

In [18]:
cnn.fit_generator(training_set, steps_per_epoch= len(training_set), validation_data = test_set, validation_steps=len(test_set), epochs = 200, callbacks = early_stopping)

C:\Users\sanja\AppData\Local\Temp/ipykernel_7240/59579697.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  cnn.fit_generator(training_set, steps_per_epoch= len(training_set), validation_data = test_set, validation_steps=len(test_set), epochs = 200, callbacks = early_stopping)


Epoch 1/200
161/161 [==============================] - 83s 507ms/step - loss: 1.0356 - accuracy: 0.4870 - val_loss: 1.0387 - val_accuracy: 0.5082
Epoch 2/200
161/161 [==============================] - 36s 224ms/step - loss: 0.9106 - accuracy: 0.5554 - val_loss: 1.0329 - val_accuracy: 0.5285
Epoch 3/200
161/161 [==============================] - 36s 226ms/step - loss: 0.8762 - accuracy: 0.5708 - val_loss: 1.0677 - val_accuracy: 0.5403
Epoch 4/200
161/161 [==============================] - 39s 243ms/step - loss: 0.8436 - accuracy: 0.5839 - val_loss: 0.9041 - val_accuracy: 0.5700
Epoch 5/200
161/161 [==============================] - 57s 354ms/step - loss: 0.8132 - accuracy: 0.6003 - val_loss: 1.0515 - val_accuracy: 0.5661
Epoch 6/200
161/161 [==============================] - 61s 377ms/step - loss: 0.7780 - accuracy: 0.6292 - val_loss: 1.0303 - val_accuracy: 0.5676
Epoch 7/200
161/161 [==============================] - 34s 210ms/step - loss: 0.7676 - accuracy: 0.6302 - val_loss: 1.1437 -

# Saving the Model

In [19]:
cnn.save('alzheimer.h5')

# Making Single prediction

## Prediction - 1

In [20]:
from keras_preprocessing.image.utils import load_img
import numpy as np
from keras.preprocessing import image

In [21]:
img = image.load_img('./Alzheimer_s Dataset/test/ModerateDemented/27 (2).jpg', target_size=(64,64))
img

In [22]:
x = image.img_to_array(img)
print(x)

[[[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 ...

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]]


In [23]:
x = np.expand_dims(x,axis=0)
print(x)

[[[[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  ...

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]]]


In [24]:
pred = np.argmax(cnn.predict(x),axis=1)
pred

array([3], dtype=int64)

In [25]:
index = ['MildDemented','ModerateDemented','NonDemented','VeryMildDemented']

In [26]:
index[pred[0]]

'VeryMildDemented'

## Prediction -2

In [27]:
img = image.load_img('./Alzheimer_s Dataset/test/NonDemented/26 (62).jpg', target_size=(64,64))
img

In [28]:
x = image.img_to_array(img)
print(x)

[[[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 ...

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]]


In [29]:
x = np.expand_dims(x,axis=0)
print(x)

[[[[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  ...

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]]]


In [30]:
pred = np.argmax(cnn.predict(x),axis=1)
pred

array([2], dtype=int64)

In [31]:
index = ['MildDemented','ModerateDemented','NonDemented','VeryMildDemented']

In [32]:
index[pred[0]]

'NonDemented'

## Prediction - 3

In [33]:
img = image.load_img('./Alzheimer_s Dataset/test/VeryMildDemented/26 (59).jpg', target_size=(64,64))
img

In [34]:
x = image.img_to_array(img)
print(x)

[[[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 ...

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]]


In [35]:
x = np.expand_dims(x,axis=0)
print(x)

[[[[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  ...

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]]]


In [36]:
pred = np.argmax(cnn.predict(x),axis=1)
pred

array([3], dtype=int64)

In [37]:
index = ['MildDemented','ModerateDemented','NonDemented','VeryMildDemented']

In [38]:
index[pred[0]]

'VeryMildDemented'

## Prediction - 4

In [39]:
img = image.load_img('./Alzheimer_s Dataset/test/MildDemented/27 (20).jpg', target_size=(64,64))
img

In [40]:
x = image.img_to_array(img)
print(x)

[[[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 ...

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]]


In [41]:
x = np.expand_dims(x,axis=0)
print(x)

[[[[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  ...

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]]]


In [42]:
pred = np.argmax(cnn.predict(x),axis=1)
pred

array([3], dtype=int64)

In [43]:
index = ['MildDemented','ModerateDemented','NonDemented','VeryMildDemented'] 

In [44]:
index[pred[0]]

'VeryMildDemented'